In [3]:
using LinearAlgebra, TimerOutputs

const BlasInt = LinearAlgebra.BlasInt

function dsyevd_workspace_query(n::Int)
    A = zeros(n, n)
    w = zeros(n)
    work = zeros(1)
    iwork = zeros(BlasInt, 1)
    info = Ref{BlasInt}(0)

    ccall((:dsyevd_64_, "libopenblas64_"), Cvoid,
        (Ref{UInt8}, Ref{UInt8}, Ref{BlasInt}, Ptr{Float64}, Ref{BlasInt},
         Ptr{Float64}, Ptr{Float64}, Ref{BlasInt}, Ptr{BlasInt}, Ref{BlasInt}, Ref{BlasInt}),
        UInt8('V'), UInt8('U'), BlasInt(n),
        pointer(A), BlasInt(n), pointer(w),
        pointer(work), BlasInt(-1), pointer(iwork), BlasInt(-1), info)

    lwork_opt = Int(work[1])
    liwork_opt = Int(iwork[1])
    return lwork_opt, liwork_opt
end

function dsyevd!(jobz::Char, uplo::Char, n::Int, A::Matrix{Float64}, lda::Int,
                 w::Vector{Float64}, work::Vector{Float64}, lwork::Int,
                 iwork::Vector{BlasInt}, liwork::Int, info::Base.RefValue{BlasInt})
    ccall((:dsyevd_64_, "libopenblas64_"), Cvoid,
          (Ref{UInt8}, Ref{UInt8}, Ref{BlasInt}, Ptr{Float64}, Ref{BlasInt},
           Ptr{Float64}, Ptr{Float64}, Ref{BlasInt}, Ptr{BlasInt}, Ref{BlasInt}, Ref{BlasInt}),
          UInt8(jobz), UInt8(uplo), BlasInt(n),
          pointer(A), BlasInt(lda), pointer(w),
          pointer(work), BlasInt(lwork), pointer(iwork), BlasInt(liwork), info)
end

function Project_onto_PSD_Decompose!(
    A::Matrix{Float64}, Aplus::Matrix{Float64}, V::Matrix{Float64}, λ::Vector{Float64},
    W::Matrix{Float64}, work::Vector{Float64}, iwork::Vector{BlasInt}, info::Base.RefValue{BlasInt},
    timer::TimerOutput = to
)
    @timeit timer "Project_onto_PSD_Decompose" begin
        n = size(A, 1)
        copyto!(V, A)
        dsyevd!('V', 'U', n, V, n, λ, work, length(work), iwork, length(iwork), info)
        fill!(Aplus, 0.0)

        @inbounds for j in 1:n
            λj = max(λ[j], 0.0)
            scale = sqrt(λj)
            for i in 1:n
                W[i, j] = V[i, j] * scale
            end
        end

        @inbounds for j in 1:n
            for i in 1:n, k in 1:n
                Aplus[i, k] += W[i, j] * W[k, j]
            end
        end
    end
end

function project_onto_UD!(A::Matrix{Float64}, timer::TimerOutput = to)
    @timeit timer "project_onto_UD!" begin
        @inbounds for i in 1:size(A, 1)
            A[i, i] = 1.0
        end
    end
    return A
end

function nearest_corr_dykstra(
    A_symm::Matrix{Float64}, tol::Float64 = 1e-8, max_iter::Int = 1000, timer::TimerOutput = to
)
    n = size(A_symm, 1)
    lwork, liwork = dsyevd_workspace_query(n)
    work = zeros(Float64, lwork)
    iwork = zeros(BlasInt, liwork)
    info = Ref{BlasInt}(0)

    Y_k = copy(A_symm)
    ΔS_psd = zeros(n, n)
    ΔS_diag = zeros(n, n)
    R_k = similar(A_symm)
    X_k = similar(A_symm)
    Aplus = similar(A_symm)
    V = similar(A_symm)
    W = similar(A_symm)
    λ = zeros(n)
    diff_matrix = similar(A_symm)

    @timeit timer "nearest_corr_dykstra" begin
        for k in 1:max_iter
            R_k .= Y_k .- ΔS_psd
            Project_onto_PSD_Decompose!(R_k, Aplus, V, λ, W, work, iwork, info, timer)
            X_k .= Aplus
            ΔS_psd .= X_k .- R_k

            R_k .= X_k .- ΔS_diag
            Y_k .= R_k
            project_onto_UD!(Y_k, timer)
            ΔS_diag .= Y_k .- R_k

            diff_matrix .= Y_k .- X_k
            rel_diff = norm(diff_matrix, Inf) / max(norm(Y_k, Inf), eps(Float64))
            if rel_diff <= tol
                println("Converged after $k iterations")
                return Y_k, k
            end
        end
        println("Reached maximum iterations ($max_iter) without convergence")
        return Y_k, max_iter
    end
end

# Example
A_symm = [
     2.0 -1.0  0.0  0.0;
    -1.0  2.0 -1.0  0.0;
     0.0 -1.0  2.0 -1.0;
     0.0  0.0 -1.0  2.0
]

println("=== Nearest Correlation via Dykstra (Zero Allocation) ===")
reset_timer!(to)
result, its = nearest_corr_dykstra(A_symm)
show(to)

min_eigenval = minimum(eigen(Symmetric(result)).values)
diagonal_check = all(abs.(diag(result) .- 1.0) .< 1e-10)
println("\nMinimum Eigenvalue after projection: ", min_eigenval)
println("Diagonal all ones: ", diagonal_check)
println("|| A - X ||_F: ", norm(A_symm - result))


=== Nearest Correlation via Dykstra (Zero Allocation) ===


LoadError: UndefVarError: `to` not defined in `Main`
Suggestion: check for spelling errors or missing imports.